Analisar a base de dados IMDB, entregue em aula. 

Utilizar como variáveis regressoras:
1. Receita (revenue)
2. Orçamento (budget)
3. Língua Original (original_language)
4. País de produção (production_coutries)
5. Data de lançamento (release_date)

Tratamento:
1. vote_count > 500
2. original_language (== EN || != EN)
3. revenue (tratamento para função exponencial)
4. budget (tratamento para função exponencial)
5. production_countries (contém EUA || não contém EUA)
6. release_date (> 2000 || < 2000)
7. Informações zeradas: cortar a linha toda!

Variável de interesse: *média da nota do filme*

Criar modelo de regressão linear e modelo de regressão logística.

Utilizar 50% da base como treino, separar da base total.
Restante das observações, utilizar 20 amostras parciais com 10% do total.

Avaliar se score de teste é estatisticamente igual ao score de treino. 

Verificar qual modelo utilizar e justificar (1 parágrafo).

Exemplo: 
Se fossem 1000 observações:
500 => somente para treino
500 => gerar 20 amostras com 10% do total (20 amostras de 50 observações)

In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

In [48]:
df = pd.read_csv('datasets/TMDB_all_movies.csv')
df.shape

(906547, 25)

In [49]:
df.head()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,...,genres,production_companies,production_countries,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer
0,2,Ariel,7.082,304,Released,1988-10-21,0.0,73,0.0,tt0094675,...,"Drama, Comedy, Romance, Crime",Villealfa Filmproductions,Finland,suomi,"Markku Rantala, Esko Nikkari, Matti Pellonpää,...",Aki Kaurismäki,Timo Salminen,Aki Kaurismäki,Aki Kaurismäki,NaN
1,3,Shadows in Paradise,7.302,344,Released,1986-10-17,0.0,74,0.0,tt0092149,...,"Drama, Comedy, Romance",Villealfa Filmproductions,Finland,"suomi, English, svenska","Esko Nikkari, Matti Pellonpää, Sirkka Silin, H...",Aki Kaurismäki,Timo Salminen,Aki Kaurismäki,Mika Kaurismäki,NaN
2,5,Four Rooms,5.800,2534,Released,1995-12-09,4257354.0,98,4000000.0,tt0113101,...,Comedy,"Miramax, A Band Apart",United States of America,English,"Amanda de Cadenet, Antonio Banderas, Madonna, ...","Alexandre Rockwell, Robert Rodriguez, Quentin ...","Guillermo Navarro, Phil Parmet, Rodrigo García...","Alexandre Rockwell, Robert Rodriguez, Quentin ...","Alexandre Rockwell, Lawrence Bender, Quentin T...",Combustible Edison
3,6,Judgment Night,6.527,316,Released,1993-10-15,12136938.0,109,21000000.0,tt0107286,...,"Action, Crime, Thriller","Largo Entertainment, JVC, Universal Pictures",United States of America,English,"Everlast, Peter Greene, Michael DeLorenzo, Den...",Stephen Hopkins,Peter Levy,"Jere Cunningham, Lewis Colick","Marilyn Vance, Gene Levy, Lloyd Segan",Alan Silvestri
4,8,Life in Loops (A Megacities RMX),7.500,27,Released,2006-01-01,0.0,80,42000.0,tt0825671,...,Documentary,inLoops,Austria,"English, हिन्दी, 日本語, Pусский, Español",NaN,Timo Novotny,Wolfgang Thaler,"Michael Glawogger, Timo Novotny","Ulrich Gehmacher, Timo Novotny",NaN


In [50]:
# Transformando os dados

df = df[df['vote_count'] > 500]

# Transformar 'original_language' em uma variável binária
df['original_language'] = df['original_language'].apply(lambda x: 1 if x == 'en' else 0)

# Usando a dica do np.log nas variáveis 'revenue' e 'budget'
df['revenue'] = df['revenue'].apply(lambda x: np.log(x) if x > 0 else 0)
df['budget'] = df['budget'].apply(lambda x: np.log(x) if x > 0 else 0)

# Transformar 'production_countries' em uma variável binária
df['production_countries'] = df['production_countries'].apply(lambda x: 1 if isinstance(x, str) and 'United States of America' in x else 0)

# Transformar 'release_date' em uma variável binária
df['release_date'] = pd.to_datetime(df['release_date']).apply(lambda x: 1 if x.year > 2000 else 0)

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6676 entries, 2 to 817429
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       6676 non-null   int64  
 1   title                    6676 non-null   object 
 2   vote_average             6676 non-null   float64
 3   vote_count               6676 non-null   int64  
 4   status                   6676 non-null   object 
 5   release_date             6676 non-null   int64  
 6   revenue                  6676 non-null   float64
 7   runtime                  6676 non-null   int64  
 8   budget                   6676 non-null   float64
 9   imdb_id                  6675 non-null   object 
 10  original_language        6676 non-null   int64  
 11  original_title           6676 non-null   object 
 12  overview                 6676 non-null   object 
 13  popularity               6676 non-null   float64
 14  tagline                  59

In [52]:
df.isnull().sum()

id                            0
title                         0
vote_average                  0
vote_count                    0
status                        0
release_date                  0
revenue                       0
runtime                       0
budget                        0
imdb_id                       1
original_language             0
original_title                0
overview                      0
popularity                    0
tagline                     754
genres                        0
production_companies         10
production_countries          0
spoken_languages              3
cast                        276
director                    268
director_of_photography     776
writers                     292
producers                   360
music_composer             1530
dtype: int64

In [53]:
df = df.dropna()

In [54]:
df.shape

(4380, 25)

In [55]:
# Dividir a base de dados em 50% treino e 50% teste
df_train, df_test = train_test_split(df, test_size=0.5, random_state=42)

# Criar 20 amostras parciais com 10% do total a partir do conjunto de teste
samples = [df_test.sample(frac=0.1) for _ in range(20)]

In [56]:
# Criei minhas variáveis de treino
X_train = df_train[['revenue', 'budget', 'original_language', 'production_countries', 'release_date', ]].to_numpy()
Y_train = df_train['vote_average'].to_numpy()

# Criei meu modelo de Regressão Linear
model_linear = LinearRegression()
model_linear.fit(X_train, Y_train)
score_linear = model_linear.score(X_train, Y_train)

# Criei meu modelo de Regressão Polinomial
poly_features = PolynomialFeatures(degree=3)
X_poly_train = poly_features.fit_transform(X_train)
model_poly = LinearRegression()
model_poly.fit(X_poly_train, Y_train)
score_polynomial = model_poly.score(X_poly_train, Y_train)

# Começar a testar o modelo
test_score_linear = []
test_score_polynomial = []
for sample in samples:
  # Criando variáveis X, Y para regressão linear
  X_test = sample[['revenue', 'budget', 'original_language', 'production_countries', 'release_date']].to_numpy()
  Y_test = sample['vote_average'].to_numpy()
  test_score_linear.append(model_linear.score(X_test, Y_test))

  # Criando variável polinomial para o X
  X_test_poly = poly_features.fit_transform(X_test)
  test_score_polynomial.append(model_poly.score(X_test_poly, Y_test))

alpha = 0.1
# Modelagem do teste de hipóteses
# H0: score de teste é igual ao score de treino
#     a minha regressão não perde poder de explicação quando testa novas variáveis
# H1: score de teste é diferente do score de treino
#     a minha regressão não explica a variável de interesse da mesma maneira que o teste.

t_stat, p_valor = stats.ttest_1samp(test_score_linear, popmean=score_linear)
print(score_linear)
print(p_valor)
if p_valor < alpha:
  print("Regressão Linear: Aceito H1 e rejeito H0")
else:
  print("Regressão Linear: Aceito H0 e rejeito H1")

t_stat, p_valor = stats.ttest_1samp(test_score_polynomial, popmean=score_polynomial)
print(score_polynomial)
print(p_valor)
if p_valor < alpha:
  print("Regressão Polinomial: Aceito H1 e rejeito H0")
else:
  print("Regressão Polinomial: Aceito H0 e rejeito H1")


0.11713567747868858
0.27514359695958596
Regressão Linear: Aceito H0 e rejeito H1
0.20987603625026674
0.24161811540380987
Regressão Polinomial: Aceito H0 e rejeito H1
